# Detection of Abnormalities in Single-Lead ECGs in the PTB-XL ECG Dataset
#### Akshay Khunte
#### CPCS 482: Current Topics in Applied Machine Learning
#### March 3, 2023

### Load Libraries
The conda environment required to import the modules required for running this model is saved to the environment.yml file in this GitHub repo, and can be built using the command line with the command "conda env create -f environment.yml", which will build the environment "tf".

In [1]:
import tensorflow as tf

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
from functools import partial
import time
import matplotlib.pyplot as plt
from scipy.ndimage import median_filter

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow import keras  # tf.keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Activation, Add, AveragePooling1D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Concatenate, Conv1D, Conv2D, Dense, Flatten
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.layers import Reshape, Input, MaxPooling2D

2023-03-03 21:52:35.942293: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 21:52:36.966064: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-03 21:52:36.966128: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-03 21:52:36.966135: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

### Load Data

To download data, use either "wget -r -N -c -np https://physionet.org/files/ptb-xl/1.0.3/" in terminal or download ZIP file from https://physionet.org/content/ptb-xl/1.0.3/

Much of the code in this section was sourced from the PTB-XL dataset release above, with a few modifications (denoted by comments)

In [2]:
import wfdb
import ast

# this function implemented by me; isolates the first lead of the signal and applies a median filtering approach to remove baseline wander
def preprocess(signal, sampling_rate):
    signal = signal.T[0] # isolates first lead
    signal = signal - median_filter(signal, size = (sampling_rate,))

    return signal

# this function is as implemented by PTB-XL publishers, but modified as denoted to use my custom preprocess function
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([preprocess(signal, sampling_rate) for signal, meta in data]) # this line is modified with the "preprocess(signal, sampling_rate)" instead of "signal" as in the inital code

    return data

# this path should be set to the directory at which this dataset is downloaded.
# If this is downloaded from the internet,the path will follow the format of the commented out path;
# if downloaded using wget, use the path that is not commented out.
# path = '~/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3/'
path = '/home/ak2532/physionet.org/files/ptb-xl/1.0.3/'
sampling_rate=100

In [3]:
# load and convert annotation data
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

In [4]:
# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)

In [5]:
# custom code; allows for usage of 2D CNN instead of 1D CNN
X = np.expand_dims(X, axis = -1)

In [6]:
# function written by me, to identify ECGs as abnormal ("TRUE", or Positive) or normal ("FALSE", or Control) based on associated SCP codes for training the model
def abnormal_ecg(row):
    if 'NORM' in row['scp_codes'].keys():
        return False # normal ECG
    else:
        return True # abnormal ECG
    
Y['abnormal_ECG'] = Y.apply(lambda row: abnormal_ecg(row), axis=1)

### Split train and test sets (random_state used for consistency)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y['abnormal_ECG'], test_size = .1, random_state = 12)

### Define Model

I developed this model from scratch for the detection of both easily diaganosable and hidden cardiovascular diseases from ECGs. More information about the model can be found at my preprint here: https://www.medrxiv.org/content/10.1101/2022.12.03.22283065v1.

The model is a small (30k param) CNN which incorporates dropout as a strategy to reduce overfitting.

In [8]:
# These are two helper functions used my the "cnn" function

def conv2d_bn_mp(x,
              filters,
              kernel_size,
              pool_size,
              strides=1,
              padding='same',
              activation='relu',
              use_bias=False,
              name=None):
    """Utility function to apply conv + BN.
    # Returns
        Output tensor after applying `Conv2D` and `BatchNormalization`.
    """
    x = Conv2D(filters,
               kernel_size,
               strides=strides,
               padding=padding,
               use_bias=use_bias,
               name=name)(x)
    # if not use_bias:
    #     bn_axis = 3
    #     bn_name = _generate_layer_name('BatchNorm', prefix=name)
    #     x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    # if activation is not None:
    #     ac_name = _generate_layer_name('Activation', prefix=name)
    #     x = Activation(activation, name=ac_name)(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = MaxPooling2D(pool_size = pool_size, padding = 'same')(x)
    return x

def fc_layer(x, units, dropout_rate = 0.2, activation = 'relu'):
    x = Dense(units)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout_rate)(x)

    return x

In [9]:
def cnn(input_shape, dropout_rate):
    # input shape: (None, Signal Length, Num Leads, 1)
    # dropout_rate: float from 0->1
    # include_top : keep prediction output layer or not

    inputs = Input(batch_shape = input_shape, name = 'input')
    x = conv2d_bn_mp(inputs, filters = 16, kernel_size = (7,1), pool_size = (2,1))
    x = conv2d_bn_mp(x, filters = 16, kernel_size = (7,1), pool_size = (2,1))
    x = conv2d_bn_mp(x, filters = 16, kernel_size = (5,1), pool_size = (2,1))
    x = conv2d_bn_mp(x, filters = 32, kernel_size = (5,1), pool_size = (4,1))
    x = conv2d_bn_mp(x, filters = 32, kernel_size = (3,1), pool_size = (2,1))
    x = conv2d_bn_mp(x, filters = 64, kernel_size = (3,1), pool_size = (2,1))
    x = conv2d_bn_mp(x, filters = 64, kernel_size = (3,1), pool_size = (4,1))

    x = fc_layer(x, 64, dropout_rate = dropout_rate)
    x = fc_layer(x, 32, dropout_rate = dropout_rate)

    x = Flatten()(x)
    outputs = Dense(1, activation = 'sigmoid', name = 'output')(x)
    model = Model(inputs, outputs, name = 'cnn')

    return model

In [10]:
# Custom Loss function which generates weights based on the specific distribution of positives/control in the dataset used for training. This was also developed on my own
class CustomLoss(tf.keras.losses.Loss):
    def __init__(self, class_names, df):
        super().__init__()

        self.weights = self.calc_weights(class_names, df)
        # self.weights = np.array([[1, 0.1]])

    def get_effective_weights(self, samples_per_class, n_classes = 2, beta = .999995):
        effective_num = 1.0-np.power(beta, samples_per_class)
        weights = (1.0 - beta) / np.array(effective_num)
        weights = weights / np.sum(weights) * n_classes
        return(weights)

    def calc_weights(self, class_names, df):    
        # class_names: labels for which weights should be calculated in format ['label1', 'label2']
        # df: pandas dataframe with columns in 'class_names' for which weights should be calculated

        weights = np.empty([len(class_names),2])
        for c in range(len(class_names)):
            one = np.count_nonzero(df[class_names[c]]==1)
            zero = np.count_nonzero(df[class_names[c]]==0)
            weights[c] = self.get_effective_weights([one,zero], beta = .999995)
            #weights[c][0] = labels.shape[0]/(2*np.count_nonzero(labels[class_names[c]]==1))
            #weights[c][1] = labels.shape[0]/(2*np.count_nonzero(labels[class_names[c]]==0))
        
        return weights

    def call(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)

        weights = self.weights
        weights = np.array(weights)
        return K.mean((weights[:,1]**(1-y_true))*(weights[:,0]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)

### Set training parameters

In [11]:
learning_rate = 2e-5
batch_size = 64
dropout = 0.5
epochs = 200

### Build and Compile Model

In [12]:
model = cnn((None, 1000, 1, 1), dropout)
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer = opt, loss=CustomLoss(['abnormal_ECG'], Y), metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC(curve='ROC'), tf.keras.metrics.AUC(curve='PR')])

2023-03-03 21:57:22.455070: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2006] Ignoring visible gpu device (device: 3, name: Quadro P600, pci bus id: 0000:05:00.0, compute capability: 6.1) with core count: 3. The minimum required count is 8. You can adjust this requirement with the env var TF_MIN_GPU_MULTIPROCESSOR_COUNT.
2023-03-03 21:57:22.461023: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 21:57:24.479442: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7759 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:06:00.0, compute capability: 6.1
2023-03-03 21:57:24.501399: I tensorflow/core/common_runtime/gp

### Finish Prepping Model

In [13]:
# requires a model_checkpoints folder, where these model checkpoints are saved; added to gitignore
save_path = f'/home/ak2532/CPSC482_CodingAssignment/model_checkpoints/LR{learning_rate}_dropout{dropout}/trained_model'
saved_model_file = save_path + '_{epoch:02d}'

# save each model epoch's weights
checkpoint = ModelCheckpoint(saved_model_file, monitor='val_loss', save_format = 'tf', save_weights_only = True, save_best_only=False, verbose=1)

# ensure that model stops training after overfitting for three epochs
early_stop = EarlyStopping(monitor='val_loss', patience=3)

# log results from each epoch to a CSV
log_path = f'/home/ak2532/CPSC482_CodingAssignment/model_checkpoints/LR{learning_rate}_dropout{dropout}/'
csv_logger = CSVLogger(log_path + f'training.csv', append = True)

callbacks = [checkpoint, early_stop, csv_logger]

### Train Model

In [15]:
history = model.fit(x=X_train,
    y=y_train,
    validation_split = 0.1,
    verbose = 1,
    epochs = epochs,
    batch_size=batch_size,
    callbacks = callbacks,
    shuffle=True,
)

## Evaluate Model on Test Set
Best epoch chosen by selecting epoch with highest val AUROC

### Evaluate Model in Held-Out Test Set

In [16]:
# This code was to evaluate the last epoch of models trained with different LRs against the test set.
# The last epoch (epoch 48) of the model trained at a LR of 2e-5 performed best
# lr = 2e-5
epoch = 48
# model.load_weights(f'/home/ak2532/CPSC482_CodingAssignment/model_checkpoints/LR{lr}_dropout0.5/trained_model_{epoch:02d}')

# these weights have been moved to this GitHub Repo, so they can be loaded for evaluation
# a copy of the associated training.csv file has also been added to the repo for training/val auroc metrics during training.
model.load_weights(f'/home/ak2532/CPSC482_CodingAssignment/trained_model_{epoch:02d}')
model.evaluate(x=X_test, y = y_test, batch_size = batch_size)

35/35 [==============================] - 0s 7ms/step - loss: 0.3959 - binary_accuracy: 0.8064 - auc: 0.8912 - auc_1: 0.9269


[0.3959450423717499,
 0.8064219951629639,
 0.8911879062652588,
 0.9268943071365356]

# REPORT

## INTRODUCTION

Artificial intelligence (AI) has been shown to diagnose many different diseases from electrocardiograms (ECGs), including diagnoses that have traditionally relied on comprehensive echocardiography or other cardiac imaging (Attia et al., 2019; Galasko et al., 2006). Even though AI-ECG is a promising screening tool for many cardiovascular conditions, most algorithms have been designed in clinically obtained 12-lead ECGs. Advances in wearable and handheld technologies now enable the point-of-care acquisition of single-lead ECG signals, paving the path for efficient and scalable AI screening tools for use with these technologies (Duarte et al., 2019; Kamga et al., 2022). This improved accessibility could enable broader AI-based screening for cardiovascular condition, and has already began implementation on wearable devices like the Apple Watch (Seshadri et al., 2020).  

This project sought to develop an algorithm using the PTB-XL dataset for the detection of abnormal single-lead ECGs, which could be deployed using wearable ECG sensors so that patients with rhythmic or conductive abnormalities could be flagged and directed to seek appropriate medical care.

## METHODS

#### Data Source

Raw voltage data for lead I was isolated from 12-lead ECGs included in PTB-XL, a large, publicly available dataset of ECGs published in 2020 (Wagner et al., 2020). Lead I was chosen as it represents the standard lead obtained from wearable devices (Duarte et al., 2019). PTB-XL includes each ECG in a 100 Hz and 500 Hz sampling frequencies. The 100 Hz sampling frequency recordings were selected for model development to allow for greater parity with wearable device ECG recordings, which have variable sampling rates and may require down sampling to reduce noise. 

#### Data Preprocessing

A standard preprocessing strategy was used to isolate signal from lead I of 12-lead ECGs, which included median pass filtering. The Lead I signal was isolated from each ECG, and a one second median filter was calculated for and subtracted from each single-lead ECG to remove baseline drift. The PTB-XL ECGs were already scaled to millivolts, which remained unchanged.

#### Outcome Label

ECGs were marked as Normal (controls), or Abnormal (positives) by searching through the SCP codes associated with each ECG in the PTB-XL dataset. If an ECG had a code “NORM”, representing a normal rhythm, in the set of SCP codes affiliated with that ECG, that ECG was labeled with a “False” label, marking it as a control. All other ECGs were labeled with a “true” label, marking it as a positive.

#### Model Training

The ECGs were randomly subset into a training/validation and a held-out test set (90%, 10%). A random seed was used to allow for split replication. A second 90%-10% split of the training/validation set was used during training to create separate training and validation sets. A CNN architecture previously reported to diagnose left ventricular systolic dysfunction with significant accuracy was used after slight modifications to accept 100 Hz ECGs (Khunte et al., 2022). This architecture consisted of a (1000, 1, 1) input layer, corresponding to a 10-second, 100 Hz, Lead I ECG, followed by seven two-dimensional convolutional layers, each of which were followed by a batch normalization layer (Ioffe & Szegedy, 07--09 Jul 2015), ReLU activation layer, and a two-dimensional max-pooling layer. The output of the seventh convolutional layer was then taken as input into a fully connected network consisting of two dense layers, each of which were followed by a batch normalization layer, ReLU activation layer, and a dropout layer with a dropout rate of 0.5 (Hinton et al., 2012). The output layer was a dense layer with one class and a sigmoid activation function. Model weights were calculated for the loss function such that learning was not affected by the lower frequency of abnormal ECGs compared to normal ECGs using the effective number of samples class re-weighting scheme. Cui et al., n.d.).  

The model was trained on the Keras framework in TensorFlow 2.9.1 and Python 3.9 using the Adam optimizer. The final model was trained at a learning rate of 0.00002 until performance on the validation set did not improve for three consecutive epochs. The epoch with the highest performance on the validation set was selected for evaluation on the test set. Other learning rates, including 0.001, 0.0001, and 0.00001 were also evaluated, but did not meet the same performance of the final model in the validation subset.

## RESULTS

The model trained to a maximum training and validation AUROC of 0.867 and 0.885, respectively. The models had a maximum training and validation AURPC of 0.912 and 0.922, respectively. This increased performance in the validation subset is attributable to the dropout layers in the model, which exclusively affect the performance in the training subset. These performance metrics generalized extremely well to the test set, in which the model outperformed its metrics in the validation set with an AUROC and AUPRC of 0.892 and 0.927, respectively. AUROC. With high AUROC and AUPRC in the held-out test set, this indicates that the model has high discrimination for distinguishing between abnormal and normal ECGs. Additionally, the high AUPRC suggests that this performance is not hindered by the imbalanced dataset, given that most ECGs in the dataset are from individuals without any ECG abnormalities. The performance in the test set exceeding the performance in the validation set indicates that the model did not overfit to the training/validation subset, and would likely generalize well to external data sources, which is imperative for any machine learning model. 

## DISCUSSION and CONCLUSIONS

This model successfully automates the detection of ECG abnormalities. Because the model was trained exclusively on Lead I ECGs with low sampling frequencies, is can be easily ported to the single-lead ECGs obtained on wearable and portable devices. Specifically, the algorithm demonstrates excellent discriminatory performance in a completely held-out test set with ECGs sampled as low as 100 Hz, making it ideal for wearable device-based screening strategies. Such models represent an extremely promising application for AI in advancing medical care and accelerating the speed at which cardiovascular diseases are identified and treated.  

The current 12-lead ECG-based models are limited to investments by health systems to incorporate tools into digital ECG repositories, and thereby limited to individuals who already seek care in those systems. In addition to the clinically indicated ECGs limiting the scope of screening, even this technology may not be available or cost-effective for smaller hospitals and clinics with limited access to digital ECGs. Wearable devices allow obtaining ECGs that are more accessible and allow for community-wide screening, an important next step in the early detection of cardiovascular diseases. 

This study is limited by the size of the PTB-XL dataset. Although 21,837 ECGs are sufficient for the development of a model, an ideal model would be developed using hundreds of thousands of samples, with train, validation, and test sets being separated on the patient level instead of simply at the record level. Furthermore, a model to be deployed on wearable ECGs would necessitate validation on the specific data collection platform of that device, to ensure compatibility with the specific sensors and acquisition environments for wearable device ECGs. The model also strictly separates ECGs as abnormal, or normal. Other models, trained on more comprehensively labeled datasets, have demonstrated the ability to distinguish with greater specificity different diseases or disease combination, including diseases that are not conventionally observable using ECGs. The SCP codes used to label these ECGs are limited to ECG-diagnosable features, and do not contain diagnoses from downstream tests that may have been performed on the patients. Reducing the need for these frequently costly and resource-consuming advanced tests would be a substantial gain for the medical system.


## REFERENCES

Attia, Z. I., Kapa, S., Lopez-Jimenez, F., McKie, P. M., Ladewig, D. J., Satam, G., Pellikka, P. A., Enriquez-Sarano, M., Noseworthy, P. A., Munger, T. M., Asirvatham, S. J., Scott, C. G., Carter, R. E., & Friedman, P. A. (2019). Screening for cardiac contractile dysfunction using an artificial intelligence–enabled electrocardiogram. Nature Medicine, 25(1), 70–74.

Cui, Jia, Lin, & Song. (n.d.). Class-balanced loss based on effective number of samples. Proceedings of the Estonian Academy of Sciences. Biology, Ecology = Eesti Teaduste Akadeemia Toimetised. Bioloogia, Okoloogia. http://openaccess.thecvf.com/content_CVPR_2019/html/Cui_Class-Balanced_Loss_Based_on_Effective_Number_of_Samples_CVPR_2019_paper.html

Duarte, R., Stainthorpe, A., Mahon, J., Greenhalgh, J., Richardson, M., Nevitt, S., Kotas, E., Boland, A., Thom, H., Marshall, T., Hall, M., & Takwoingi, Y. (2019). Lead-I ECG for detecting atrial fibrillation in patients attending primary care with an irregular pulse using single-time point testing: A systematic review and economic evaluation. PloS One, 14(12), e0226671.

Galasko, G. I. W., Barnes, S. C., Collinson, P., Lahiri, A., & Senior, R. (2006). What is the most cost-effective strategy to screen for left ventricular systolic dysfunction: natriuretic peptides, the electrocardiogram, hand-held echocardiography, traditional echocardiography, or their combination? European Heart Journal, 27(2), 193–200.

Hinton, G. E., Srivastava, N., Krizhevsky, A., Sutskever, I., & Salakhutdinov, R. R. (2012). Improving neural networks by preventing co-adaptation of feature detectors. In arXiv [cs.NE]. arXiv. http://arxiv.org/abs/1207.0580

Ioffe, S., & Szegedy, C. (07--09 Jul 2015). Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift. In F. Bach & D. Blei (Eds.), Proceedings of the 32nd International Conference on Machine Learning (Vol. 37, pp. 448–456). PMLR.
Kamga, P., Mostafa, R., & Zafar, S. (2022). The Use of Wearable ECG Devices in the Clinical Setting: a Review. Current Emergency and Hospital Medicine Reports, 10(3), 67–72.

Khunte, A., Sangha, V., Oikonomou, E. K., Dhingra, L. S., Aminorroaya, A., Mortazavi, B. J., Coppi, A., Krumholz, H. M., & Khera, R. (2022). Detection of left ventricular systolic dysfunction from single-lead electrocardiography adapted for wearable devices. In bioRxiv. https://doi.org/10.1101/2022.12.03.22283065

Seshadri, D. R., Bittel, B., Browsky, D., Houghtaling, P., Drummond, C. K., Desai, M. Y., & Gillinov, A. M. (2020). Accuracy of Apple Watch for Detection of Atrial Fibrillation. Circulation, 141(8), 702–703.

Wagner, P., Strodthoff, N., Bousseljot, R.-D., Kreiseler, D., Lunze, F. I., Samek, W., & Schaeffter, T. (2020). PTB-XL, a large publicly available electrocardiography dataset. In Scientific Data (Vol. 7, Issue 1). https://doi.org/10.1038/s41597-020-0495-6 
